In [1]:
    #!/usr/bin/env python
# coding: utf-8

import dask_cudf #as dask
from dask import delayed
from dask.distributed import Client

import dask.dataframe as dd
import json
import numpy as np
import pandas as pd
import geopandas
from shapely.geometry import Point
import os
import sys
import joblib

In [2]:
dtype_list = {
              'dropoff_datetime': 'str',  # object, # set by parse_dates in pandas read_csv
              'dropoff_latitude': np.float64,
              'dropoff_taxizone_id': np.float64,
              'dropoff_longitude': np.float64,
              'ehail_fee': np.float64,
              'extra': np.float64,
              'fare_amount': np.float64,
              'improvement_surcharge': np.float64,
              'junk1': 'str',  # object,
              'junk2': 'str',  # object,
              'mta_tax': np.float64,
              'passenger_count': 'str',  # object,
              'payment_type': 'str',  # object,
              'pickup_datetime': 'str',  # object, # set by parse_dates in pandas read_csv
              'pickup_latitude': np.float64,
              'pickup_taxizone_id': np.float64,
              'pickup_longitude': np.float64,
              'rate_code_id': 'str',  # object,
              'store_and_fwd_flag': 'str',  # object,
              'tip_amount': np.float64,
              'tolls_amount': np.float64,
              'total_amount': np.float64,
              'trip_distance': np.float64,
              'trip_type': 'str',  # object,
              'vendor_id': 'str',  # object,
             }

In [3]:
# with open('config.json', 'r') as fh:
#     config = json.load(fh)

In [4]:
# make dict of paths to data directories
relative_path = '../00_download_scripts/raw_data'

config = {
          'citibike_raw_data_path': f'{relative_path}/bike/',
          'taxi_raw_data_path': f'{relative_path}/taxi/',
          'uber_raw_data_path': f'{relative_path}/uber/',
          'subway_raw_data_path': f'{relative_path}/subway/',
          'parquet_output_path': f'data/'
         }



In [5]:
# import cudf

# cudf.read_csv('../00_download_scripts/raw_data/taxi/yellow_tripdata_2009-01.csv')

In [6]:
def glob(x):
    '''
    Signature: sorted(glob(pathname=x, *, recursive=False))
    Docstring:
    Return a list of paths matching a pathname pattern.

    The pattern may contain simple shell-style wildcards a la
    fnmatch. However, unlike fnmatch, filenames starting with a
    dot are special cases that are not matched by '*' and '?'
    patterns.

    If recursive is true, the pattern '**' will match any files and
    zero or more directories and subdirectories.
    '''
    from glob import glob
    return sorted(glob(x))


def trymakedirs(path):
    '''
    Signature: try: os.makedirs(name=path, mode=511, exist_ok=False)
    Docstring:
    makedirs(name [, mode=0o777][, exist_ok=False])

    Super-mkdir; create a leaf directory and all intermediate ones.  Works like
    mkdir, except that any intermediate path segment (not just the rightmost)
    will be created if it does not exist. If the target directory already
    exists, raise an OSError if exist_ok is False. Otherwise no exception is
    raised.  This is recursive.
    '''
    try:
        os.makedirs(path)
    except:
        pass


# # cuspatial may be only single gpu
# # can we extract 1 partition at a time from a dask_cudf dataframe?
# def assign_taxi_zones(df, lon_var, lat_var, locid_var):
#     """
#     Joins DataFrame with Taxi Zones shapefile.
#     This function takes longitude values provided by `lon_var`, and latitude
#     values provided by `lat_var` in DataFrame `df`, and performs a spatial join
#     with the NYC taxi_zones shapefile. 
#     The shapefile is hard coded in, as this function makes a hard assumption of
#     latitude and longitude coordinates. It also assumes latitude=0 and 
#     longitude=0 is not a datapoint that can exist in your dataset. Which is 
#     reasonable for a dataset of New York, but bad for a global dataset.
#     Only rows where `df.lon_var`, `df.lat_var` are reasonably near New York,
#     and `df.locid_var` is set to np.nan are updated. 
#     Parameters
#     ----------
#     df : pandas.DataFrame or dask.DataFrame
#         DataFrame containing latitudes, longitudes, and location_id columns.
#     lon_var : string
#         Name of column in `df` containing longitude values. Invalid values 
#         should be np.nan.
#     lat_var : string
#         Name of column in `df` containing latitude values. Invalid values 
#         should be np.nan
#     locid_var : string
#         Name of column in `df` containing taxi_zone location ids. Rows with
#         valid, nonzero values are not overwritten. 
#         """
#     import geopandas
#     from shapely.geometry import Point


#     localdf = df[[lon_var, lat_var, locid_var]].copy()
#     # localdf = localdf.reset_index()
#     localdf[lon_var] = localdf[lon_var].fillna(value=0.)
#     localdf[lat_var] = localdf[lat_var].fillna(value=0.)
#     localdf['replace_locid'] = (localdf[locid_var].isnull()
#                                 & (localdf[lon_var] != 0.)
#                                 & (localdf[lat_var] != 0.))

#     if (np.any(localdf['replace_locid'])):
#         shape_df = geopandas.read_file('../shapefiles/taxi_zones.shp')
#         shape_df.drop(['OBJECTID', "Shape_Area", "Shape_Leng", "borough", "zone"],
#                       axis=1, inplace=True)
#         shape_df = shape_df.to_crs({'init': 'epsg:4326'})

#         try:
#             local_gdf = geopandas.GeoDataFrame(
#                 localdf, crs={'init': 'epsg:4326'},
#                 geometry=[Point(xy) for xy in
#                           zip(localdf[lon_var], localdf[lat_var])])

#             local_gdf = geopandas.sjoin(
#                 local_gdf, shape_df, how='left', op='within')

#             # one point can intersect more than one zone -- for example if on
#             # the boundary between two zones. Deduplicate by taking first valid.
#             local_gdf = local_gdf[~local_gdf.index.duplicated(keep='first')]

#             local_gdf.LocationID.values[~local_gdf.replace_locid] = (
#                 (local_gdf[locid_var])[~local_gdf.replace_locid]).values

#             return local_gdf.LocationID.rename(locid_var)
#         except ValueError as ve:
#             print(ve)
#             print(ve.stacktrace())
#             return df[locid_var].astype(np.float64)
#     else:areas
#         return df[locid_var]

In [7]:
import cuspatial
import dask_cudf
import numpy as np

# taxi2016 = dask_cudf.read_csv("taxi2016.csv")
# taxi2017 = dask_cudf.read_csv("taxi2017.csv")

# 31 polygon limit, 263 LocationID features
pip_iterations = list(np.arange(0, 263, 31))
pip_iterations.append(263)

taxi_zones = cuspatial.read_polygon_shapefile('zones/cu_taxi_zones.shp')

def assign_taxi_zones(df, lon_var, lat_var, locid_var):
    localdf = df[[lon_var, lat_var]].copy()
    # localdf = localdf.reset_index()
    localdf[lon_var] = localdf[lon_var].fillna(value=0.)
    localdf[lat_var] = localdf[lat_var].fillna(value=0.)

#     localdf['replace_locid'] = (localdf[locid_var].isnull()
#                                 & (localdf[lon_var] != 0.)
#                                 & (localdf[lat_var] != 0.))
    
    # generic pickup/dropoff location id columns
    localdf[locid_var] = 264
    
    for i in range(len(pip_iterations)-1):
        start = pip_iterations[i]
        end = pip_iterations[i+1]
        # derive taxi zones from coordinates
        pickups = cuspatial.point_in_polygon(localdf[lon_var], 
                                             localdf[lat_var], 
                                             taxi_zones[0][start:end], 
                                             taxi_zones[1], 
                                             taxi_zones[2]['x'], 
                                             taxi_zones[2]['y'])
        # correct location id columns
        for j in pickups.columns:
            localdf[locid_var].loc[pickups[j]] = j
        
    return localdf[locid_var]

# taxi2016['PULocationID'] = taxi2016.map_partitions(assign_taxi_zones, 
#                                                    'pickup_longitude', 
#                                                    'pickup_latitude', 
#                                                    'PULocationID', 
#                                                    meta=('PULocationID', np.float64))

# taxi2016['DOLocationID'] = taxi2016.map_partitions(assign_taxi_zones, 
#                                                    'pickup_longitude', 
#                                                    'pickup_latitude', 
#                                                    'DOLocationID', 
#                                                    meta=('DOLocationID', np.float64))

In [8]:
# taxi_zones = cuspatial.read_polygon_shapefile('cu_taxi_zones.shp')

# pip_iterations = list(np.arange(0, 263, 31))
# pip_iterations.append(263)
# # print(pip_iterations)

# taxi2016['PULocationID'] = 264
# taxi2016['DOLocationID'] = 264
# for i in range(len(pip_iterations)-1):
#     start = pip_iterations[i]
#     end = pip_iterations[i+1]
#     pickups = cuspatial.point_in_polygon(taxi2016['pickup_longitude'], 
#                                          taxi2016['pickup_latitude'], 
#                                          taxi_zones[0][start:end], 
#                                          taxi_zones[1], 
#                                          taxi_zones[2]['x'], 
#                                          taxi_zones[2]['y'])
#     dropoffs = cuspatial.point_in_polygon(taxi2016['dropoff_longitude'], 
#                                           taxi2016['dropoff_latitude'], 
#                                           taxi_zones[0][start:end], 
#                                           taxi_zones[1], 
#                                           taxi_zones[2]['x'], 
#                                           taxi_zones[2]['y'])
#     for j in pickups.columns:
#         taxi2016['PULocationID'].loc[pickups[j]] = j
#     for j in dropoffs.columns:
#         taxi2016['DOLocationID'].loc[dropoffs[j]] = j

In [9]:
yellow_schema_pre_2015 = "vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code_id,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,total_amount"
yellow_glob_pre_2015 = glob(
    os.path.join(config['taxi_raw_data_path'], 'yellow_tripdata_201[0-4]*.csv')) + glob(
    os.path.join(config['taxi_raw_data_path'], 'yellow_tripdata_2009*.csv'))

yellow_schema_2015_2016_h1 = "vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code_id,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount"
yellow_glob_2015_2016_h1 = glob(os.path.join(config['taxi_raw_data_path'], 'yellow_tripdata_2015*.csv')) + glob(
    os.path.join(config['taxi_raw_data_path'], 'yellow_tripdata_2016-0[1-6].csv'))

yellow_schema_2016_h2 = "vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,rate_code_id,store_and_fwd_flag,pickup_taxizone_id,dropoff_taxizone_id,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,junk1,junk2"
yellow_glob_2016_h2 = glob(os.path.join(config['taxi_raw_data_path'], 'yellow_tripdata_2016-0[7-9].csv')) + glob(
    os.path.join(config['taxi_raw_data_path'], 'yellow_tripdata_2016-1[0-2].csv'))

In [10]:
# import cudf
# for i in yellow_glob_pre_2015 + yellow_glob_2015_2016_h1 + yellow_glob_2016_h2:
#     df = cudf.read_csv(i)
#     df.to_pandas().sample(100000).to_csv(i, index=False)

In [11]:
# df = pd.read_csv(i)

# print(type(df))

# df = cudf.DataFrame().from_pandas(df)

# print(type(df))

In [12]:
# yellow1 = dd.read_csv(yellow_glob_pre_2015, header=0,
#                       na_values=["NA"],
#                       parse_dates=[1, 2],
#                       infer_datetime_format=True,
#                       dtype=dtype_list,
#                       names=yellow_schema_pre_2015.split(','))

In [13]:
# yellow1['dropoff_taxizone_id'] = yellow1['total_amount'].copy()
# yellow1['dropoff_taxizone_id'] = np.nan
# yellow1['pickup_taxizone_id'] = yellow1['total_amount'].copy()
# yellow1['pickup_taxizone_id'] = np.nan
# yellow1['ehail_fee'] = yellow1['total_amount'].copy()
# yellow1['ehail_fee'] = np.nan
# yellow1['improvement_surcharge'] = yellow1['total_amount'].copy()
# yellow1['improvement_surcharge'] = np.nan
# yellow1['trip_type'] = yellow1['rate_code_id'].copy()
# yellow1['trip_type'] = -999

In [14]:
# # January 2015 - June 2016 dataframe
# yellow2 = dd.read_csv(yellow_glob_2015_2016_h1, header=0,
#                       na_values=["NA"],
#                       parse_dates=[1, 2],
#                       infer_datetime_format=True,
#                       dtype=dtype_list,
#                       names=yellow_schema_2015_2016_h1.split(','))
# yellow2['dropoff_taxizone_id'] = yellow2['rate_code_id'].copy()
# yellow2['dropoff_taxizone_id'] = np.nan
# yellow2['pickup_taxizone_id'] = yellow2['rate_code_id'].copy()
# yellow2['pickup_taxizone_id'] = np.nan
# yellow2['ehail_fee'] = yellow2['total_amount'].copy()
# yellow2['ehail_fee'] = np.nan
# yellow2['trip_type'] = yellow2['rate_code_id'].copy()
# yellow2['trip_type'] = -999

In [15]:
def get_yellow():
    # tag file paths to data and column names by schema
    yellow_schema_pre_2015 = "vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code_id,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,total_amount"
    yellow_glob_pre_2015 = glob(
        os.path.join(config['taxi_raw_data_path'], 'yellow_tripdata_201[0-4]*.csv')) + glob(
        os.path.join(config['taxi_raw_data_path'], 'yellow_tripdata_2009*.csv'))

    yellow_schema_2015_2016_h1 = "vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code_id,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount"
    yellow_glob_2015_2016_h1 = glob(os.path.join(config['taxi_raw_data_path'], 'yellow_tripdata_2015*.csv')) + glob(
        os.path.join(config['taxi_raw_data_path'], 'yellow_tripdata_2016-0[1-6].csv'))

    yellow_schema_2016_h2 = "vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,rate_code_id,store_and_fwd_flag,pickup_taxizone_id,dropoff_taxizone_id,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,junk1,junk2"
    yellow_glob_2016_h2 = glob(os.path.join(config['taxi_raw_data_path'], 'yellow_tripdata_2016-0[7-9].csv')) + glob(
        os.path.join(config['taxi_raw_data_path'], 'yellow_tripdata_2016-1[0-2].csv'))

    # pre 2015 dataframe
    yellow1 = dask_cudf.read_csv(yellow_glob_pre_2015, 
#                                  header=0,
#                                  na_values=["NA"],
#                                  parse_dates=[1, 2],
#                                  infer_datetime_format=True,
#                                  dtype=dtype_list,
                                 names=yellow_schema_pre_2015.split(',')
                                )
    
    yellow1['dropoff_taxizone_id'] = yellow1['total_amount'].copy()
    yellow1['dropoff_taxizone_id'] = np.nan
    yellow1['pickup_taxizone_id'] = yellow1['total_amount'].copy()
    yellow1['pickup_taxizone_id'] = np.nan
    yellow1['ehail_fee'] = yellow1['total_amount'].copy()
    yellow1['ehail_fee'] = np.nan
    yellow1['improvement_surcharge'] = yellow1['total_amount'].copy()
    yellow1['improvement_surcharge'] = np.nan
    yellow1['trip_type'] = yellow1['rate_code_id'].copy()
    yellow1['trip_type'] = -999
    
    # January 2015 - June 2016 dataframe
    yellow2 = dask_cudf.read_csv(yellow_glob_2015_2016_h1, 
#                                  header=0,
#                                  na_values=["NA"], 
#                                  parse_dates=[1, 2],
#                                  infer_datetime_format=True,
#                                  dtype=dtype_list,
                                 names=yellow_schema_2015_2016_h1.split(',')
                                )
    yellow2['dropoff_taxizone_id'] = yellow2['rate_code_id'].copy()
    yellow2['dropoff_taxizone_id'] = np.nan
    yellow2['pickup_taxizone_id'] = yellow2['rate_code_id'].copy()
    yellow2['pickup_taxizone_id'] = np.nan
    yellow2['ehail_fee'] = yellow2['total_amount'].copy()
    yellow2['ehail_fee'] = np.nan
    yellow2['trip_type'] = yellow2['rate_code_id'].copy()
    yellow2['trip_type'] = -999

#     yellow3 = dd.read_csv(yellow_glob_2016_h2, header=0,
#                           na_values=["NA"],
#                           parse_dates=[1, 2],
#                           infer_datetime_format=True,
#                           dtype=dtype_list,
#                           names=yellow_schema_2016_h2.split(','))
#     yellow3['dropoff_latitude'] = yellow3['total_amount'].copy()
#     yellow3['dropoff_latitude'] = np.nan
#     yellow3['dropoff_longitude'] = yellow3['total_amount'].copy()
#     yellow3['dropoff_longitude'] = np.nan
#     yellow3['pickup_latitude'] = yellow3['total_amount'].copy()
#     yellow3['pickup_latitude'] = np.nan
#     yellow3['pickup_longitude'] = yellow3['total_amount'].copy()
#     yellow3['pickup_longitude'] = np.nan
#     yellow3['ehail_fee'] = yellow3['total_amount'].copy()
#     yellow3['ehail_fee'] = np.nan
#     yellow3['trip_type'] = yellow3['rate_code_id'].copy()
#     yellow3['trip_type'] = -999
#     yellow3 = yellow3.drop(['junk1', 'junk2'], axis=1)

    yellow = yellow1[sorted(yellow1.columns)].append(
        yellow2[sorted(yellow1.columns)])
    #yellow = yellow.append(yellow3[sorted(yellow1.columns)])

    for field in list(yellow.columns):
        if field in dtype_list:
            yellow[field] = yellow[field].astype(dtype_list[field])

    yellow['trip_type'] = 'yellow'

    return yellow

In [16]:
# yellow_schema_2015_2016_h1 = "vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code_id,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount"
# yellow_glob_2015_2016_h1 = glob(os.path.join(config['taxi_raw_data_path'], 'yellow_tripdata_2015*.csv')) + glob(
#     os.path.join(config['taxi_raw_data_path'], 'yellow_tripdata_2016-0[1-6].csv'))

# yellow_glob_2015_2016_h1

In [17]:
"""
Signature: all_trips.map_partitions(func, *args, **kwargs)
Docstring:
Apply Python function on each DataFrame partition.

Note that the index and divisions are assumed to remain unchanged.

Parameters
----------
func : function
    Function applied to each partition.
args, kwargs :
    Arguments and keywords to pass to the function. The partition will
    be the first argument, and these will be passed *after*. Arguments
    and keywords may contain ``Scalar``, ``Delayed`` or regular
    python objects. DataFrame-like args (both dask and pandas) will be
    repartitioned to align (if necessary) before applying the function.
meta : pd.DataFrame, pd.Series, dict, iterable, tuple, optional
    An empty ``pd.DataFrame`` or ``pd.Series`` that matches the dtypes
    and column names of the output. This metadata is necessary for
    many algorithms in dask dataframe to work.  For ease of use, some
    alternative inputs are also available. Instead of a ``DataFrame``,
    a ``dict`` of ``{name: dtype}`` or iterable of ``(name, dtype)``
    can be provided (note that the order of the names should match the
    order of the columns). Instead of a series, a tuple of ``(name,
    dtype)`` can be used. If not provided, dask will try to infer the
    metadata. This may lead to unexpected results, so providing
    ``meta`` is recommended. For more information, see
    ``dask.dataframe.utils.make_meta``.

Examples
--------
Given a DataFrame, Series, or Index, such as:

>>> import dask.dataframe as dd
>>> df = pd.DataFrame({'x': [1, 2, 3, 4, 5],
...                    'y': [1., 2., 3., 4., 5.]})
>>> ddf = dd.from_pandas(df, npartitions=2)

One can use ``map_partitions`` to apply a function on each partition.
Extra arguments and keywords can optionally be provided, and will be
passed to the function after the partition.

Here we apply a function with arguments and keywords to a DataFrame,
resulting in a Series:

>>> def myadd(df, a, b=1):
...     return df.x + df.y + a + b
>>> res = ddf.map_partitions(myadd, 1, b=2)
>>> res.dtype
dtype('float64')

By default, dask tries to infer the output metadata by running your
provided function on some fake data. This works well in many cases, but
can sometimes be expensive, or even fail. To avoid this, you can
manually specify the output metadata with the ``meta`` keyword. This
can be specified in many forms, for more information see
``dask.dataframe.utils.make_meta``.

Here we specify the output is a Series with no name, and dtype
``float64``:

>>> res = ddf.map_partitions(myadd, 1, b=2, meta=(None, 'f8'))

Here we map a function that takes in a DataFrame, and returns a
DataFrame with a new column:

>>> res = ddf.map_partitions(lambda df: df.assign(z=df.x * df.y))
>>> res.dtypes
x      int64
y    float64
z    float64
dtype: object

As before, the output metadata can also be specified manually. This
time we pass in a ``dict``, as the output is a DataFrame:

>>> res = ddf.map_partitions(lambda df: df.assign(z=df.x * df.y),
...                          meta={'x': 'i8', 'y': 'f8', 'z': 'f8'})

In the case where the metadata doesn't change, you can also pass in
the object itself directly:

>>> res = ddf.map_partitions(lambda df: df.head(), meta=ddf)

Also note that the index and divisions are assumed to remain unchanged.
If the function you're mapping changes the index/divisions, you'll need
to clear them afterwards:

>>> ddf.map_partitions(func).clear_divisions()  # doctest: +SKIP
File:      /opt/conda-environments/rapids-stable/lib/python3.7/site-packages/dask/dataframe/core.py
Type:      method
"""
pass

In [18]:
# assign_taxi_zones

In [19]:
# yellow = get_yellow()

# #     all_trips = uber.append(green).append(yellow)
# all_trips = yellow

# all_trips['dropoff_taxizone_id'] = all_trips.map_partitions(
#                                                             assign_taxi_zones, 
#                                                             "dropoff_longitude", 
#                                                             "dropoff_latitude",
#                                                             "dropoff_taxizone_id", 
#                                                             meta=('dropoff_taxizone_id', np.float64)
#                                                             )

# START BREAK DOWN

In [20]:
yellow = get_yellow()
type(yellow)

dask_cudf.core.DataFrame

In [21]:
#     all_trips = uber.append(green).append(yellow)
all_trips = yellow

In [22]:
all_trips['dropoff_taxizone_id'] = all_trips.map_partitions(
                                                            assign_taxi_zones, 
                                                            lon_var='dropoff_longitude', 
                                                            lat_var='dropoff_latitude',
                                                            locid_var='dropoff_taxizone_id', 
                                                            meta=('dropoff_taxizone_id', np.float64))

In [23]:
all_trips.tail()

,dropoff_datetime,dropoff_latitude,dropoff_longitude,dropoff_taxizone_id,ehail_fee,extra,fare_amount,improvement_surcharge,mta_tax,passenger_count,...,pickup_longitude,pickup_taxizone_id,rate_code_id,store_and_fwd_flag,tip_amount,tolls_amount,total_amount,trip_distance,trip_type,vendor_id
624276,2016-01-31 23:38:18,40.767509,-73.982651,47,NaN,0.5,8.5,0.3,0.5,1,...,-74.003578,-9.223372e+18,1,N,0.00,0.00,9.80,2.2,yellow,2
624277,2016-01-05 00:16:06,40.751530,-73.945457,148,NaN,0.5,2.5,0.3,0.5,1,...,-73.945488,-9.223372e+18,1,N,0.00,0.00,3.80,0.0,yellow,1
624278,2016-03-19 20:45:50,40.753922,-73.984428,163,NaN,0.5,7.5,0.3,0.5,3,...,-73.994240,-9.223372e+18,1,N,0.00,0.00,8.80,1.4,yellow,1
624279,2016-03-28 12:54:26,40.777435,-73.978188,143,NaN,0.0,11.5,0.3,0.5,1,...,-73.948067,-9.223372e+18,1,N,2.45,0.00,14.75,2.1,yellow,1
624280,2016-01-05 06:15:36,40.758583,-73.961006,228,NaN,0.0,52.0,0.3,0.5,3,...,-73.960938,-9.223372e+18,2,N,0.00,5.54,58.34,0.0,yellow,1


In [24]:
all_trips['pickup_taxizone_id'] = all_trips.map_partitions(
                                                           assign_taxi_zones, 
                                                           lon_var='pickup_longitude', 
                                                           lat_var='pickup_latitude',
                                                           locid_var='pickup_taxizone_id', 
                                                           meta=('pickup_taxizone_id', np.float64))

In [25]:
all_trips = all_trips[sorted(all_trips.columns)]
# all_trips = all_trips.repartition(npartitions=1200)

In [26]:
all_trips.tail()

,dropoff_datetime,dropoff_latitude,dropoff_longitude,dropoff_taxizone_id,ehail_fee,extra,fare_amount,improvement_surcharge,mta_tax,passenger_count,...,pickup_longitude,pickup_taxizone_id,rate_code_id,store_and_fwd_flag,tip_amount,tolls_amount,total_amount,trip_distance,trip_type,vendor_id
624276,2016-01-31 23:38:18,40.767509,-73.982651,47,NaN,0.5,8.5,0.3,0.5,1,...,-74.003578,248,1,N,0.00,0.00,9.80,2.2,yellow,2
624277,2016-01-05 00:16:06,40.751530,-73.945457,148,NaN,0.5,2.5,0.3,0.5,1,...,-73.945488,148,1,N,0.00,0.00,3.80,0.0,yellow,1
624278,2016-03-19 20:45:50,40.753922,-73.984428,163,NaN,0.5,7.5,0.3,0.5,3,...,-73.994240,49,1,N,0.00,0.00,8.80,1.4,yellow,1
624279,2016-03-28 12:54:26,40.777435,-73.978188,143,NaN,0.0,11.5,0.3,0.5,1,...,-73.948067,262,1,N,2.45,0.00,14.75,2.1,yellow,1
624280,2016-01-05 06:15:36,40.758583,-73.961006,228,NaN,0.0,52.0,0.3,0.5,3,...,-73.960938,228,2,N,0.00,5.54,58.34,0.0,yellow,1


In [27]:
all_trips = all_trips.map_partitions(lambda x: x.sort_values('pickup_datetime'), 
    meta=all_trips)

In [28]:
all_trips.tail()

,dropoff_datetime,dropoff_latitude,dropoff_longitude,dropoff_taxizone_id,ehail_fee,extra,fare_amount,improvement_surcharge,mta_tax,passenger_count,...,pickup_longitude,pickup_taxizone_id,rate_code_id,store_and_fwd_flag,tip_amount,tolls_amount,total_amount,trip_distance,trip_type,vendor_id
624272,2016-01-31 22:01:58,40.656689,-73.958153,189,NaN,0.5,29.0,0.3,0.5,1,...,-74.002953,248,1,N,5.00,0.0,35.30,7.83,yellow,2
624273,2016-01-31 22:45:04,40.750488,-73.994637,185,NaN,0.5,9.5,0.3,0.5,1,...,-74.009277,230,1,N,2.16,0.0,12.96,2.50,yellow,2
624274,2016-01-31 22:59:37,40.734909,-74.002159,251,NaN,0.5,7.0,0.3,0.5,1,...,-74.003578,248,1,N,1.00,0.0,9.30,1.68,yellow,2
624275,2016-01-31 23:12:08,40.761669,-73.999680,49,NaN,0.5,11.0,0.3,0.5,1,...,-74.002159,251,1,N,1.00,0.0,13.30,2.65,yellow,2
624276,2016-01-31 23:38:18,40.767509,-73.982651,47,NaN,0.5,8.5,0.3,0.5,1,...,-74.003578,248,1,N,0.00,0.0,9.80,2.20,yellow,2


In [29]:
for fieldName in all_trips.columns:
    if fieldName in dtype_list:
        all_trips[fieldName] = all_trips[fieldName].astype(dtype_list[fieldName])

In [30]:
all_trips.tail()

,dropoff_datetime,dropoff_latitude,dropoff_longitude,dropoff_taxizone_id,ehail_fee,extra,fare_amount,improvement_surcharge,mta_tax,passenger_count,...,pickup_longitude,pickup_taxizone_id,rate_code_id,store_and_fwd_flag,tip_amount,tolls_amount,total_amount,trip_distance,trip_type,vendor_id
624272,2016-01-31 22:01:58,40.656689,-73.958153,189.0,NaN,0.5,29.0,0.3,0.5,1,...,-74.002953,248.0,1,N,5.00,0.0,35.30,7.83,yellow,2
624273,2016-01-31 22:45:04,40.750488,-73.994637,185.0,NaN,0.5,9.5,0.3,0.5,1,...,-74.009277,230.0,1,N,2.16,0.0,12.96,2.50,yellow,2
624274,2016-01-31 22:59:37,40.734909,-74.002159,251.0,NaN,0.5,7.0,0.3,0.5,1,...,-74.003578,248.0,1,N,1.00,0.0,9.30,1.68,yellow,2
624275,2016-01-31 23:12:08,40.761669,-73.999680,49.0,NaN,0.5,11.0,0.3,0.5,1,...,-74.002159,251.0,1,N,1.00,0.0,13.30,2.65,yellow,2
624276,2016-01-31 23:38:18,40.767509,-73.982651,47.0,NaN,0.5,8.5,0.3,0.5,1,...,-74.003578,248.0,1,N,0.00,0.0,9.80,2.20,yellow,2


In [31]:
all_trips.tail(250000).to_parquet('sample_taxi.parquet')

In [32]:
import cudf
cudf.read_parquet('sample_taxi.parquet')

,dropoff_datetime,dropoff_latitude,dropoff_longitude,dropoff_taxizone_id,ehail_fee,extra,fare_amount,improvement_surcharge,mta_tax,passenger_count,...,pickup_longitude,pickup_taxizone_id,rate_code_id,store_and_fwd_flag,tip_amount,tolls_amount,total_amount,trip_distance,trip_type,vendor_id
0,2016-02-12 11:42:29,40.768520,-73.952156,138.0,NaN,0.5,15.5,0.3,0.5,4,...,-73.986290,73.0,2,Y,2.20,0.0,19.00,4.07,yellow,1
1,2016-01-05 00:16:06,40.718910,-74.010391,230.0,NaN,0.5,25.5,0.3,0.5,1,...,-73.952560,236.0,1,N,5.36,0.0,32.16,7.31,yellow,1
2,2016-01-05 06:12:31,40.754681,-73.853783,56.0,NaN,0.5,9.5,0.3,0.5,1,...,-73.872871,145.0,1,N,4.00,0.0,14.80,2.46,yellow,1
3,2016-03-19 20:45:50,40.779633,-73.953598,236.0,NaN,0.5,9.0,0.3,0.5,3,...,-73.965919,228.0,1,N,2.06,0.0,12.36,2.10,yellow,1
4,2016-02-11 15:06:37,40.763000,-73.971962,162.0,NaN,0.5,9.5,0.3,0.5,1,...,-73.990799,185.0,1,N,0.00,0.0,10.80,1.86,yellow,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,2016-01-09 10:57:23,40.656689,-73.958153,189.0,NaN,0.5,29.0,0.3,0.5,2,...,-74.002953,248.0,1,N,5.00,0.0,35.30,7.83,yellow,2
249996,2016-01-09 10:53:14,40.750488,-73.994637,185.0,NaN,0.5,9.5,0.3,0.5,5,...,-74.009277,230.0,1,N,2.16,0.0,12.96,2.50,yellow,2
249997,2016-01-09 10:54:19,40.734909,-74.002159,251.0,NaN,0.5,7.0,0.3,0.5,6,...,-74.003578,248.0,1,N,1.00,0.0,9.30,1.68,yellow,2
249998,2016-01-09 10:53:42,40.761669,-73.999680,49.0,NaN,0.5,11.0,0.3,0.5,1,...,-74.002159,251.0,1,N,1.00,0.0,13.30,2.65,yellow,2


In [33]:
all_trips.compute()#.to_parquet('all_trips_unprocessed.parquet')

ValueError: Could not convert strings to float                         type due to presence of non-floating values.

In [ ]:
all_trips.to_parquet('all_trips_unprocessed.parquet')

In [ ]:
# all_trips.to_parquet(
#     os.path.join(config['parquet_output_path'], 'all_trips_unprocessed.parquet'),
#     compression='GZIP', 
#         has_nulls=True,
#         object_encoding='json'
# )


# END BREAK DOWN

In [ ]:
def main(client):

#     uber = get_uber()
#     green = get_green()
    yellow = get_yellow()

#     all_trips = uber.append(green).append(yellow)
    all_trips = yellow

    all_trips['dropoff_taxizone_id'] = all_trips.map_partitions(
                                                                assign_taxi_zones, 
                                                                lon_var='dropoff_longitude', 
                                                                lat_var='dropoff_latitude',
                                                                locid_var='dropoff_taxizone_id', 
                                                                meta=('dropoff_taxizone_id', np.float64))
    all_trips['pickup_taxizone_id'] = all_trips.map_partitions(
                                                               assign_taxi_zones, 
                                                               lon_var='pickup_longitude', 
                                                               lat_var='pickup_latitude',
                                                               locid_var='pickup_taxizone_id', 
                                                               meta=('pickup_taxizone_id', np.float64))

    all_trips = all_trips[sorted(all_trips.columns)]
    # all_trips = all_trips.repartition(npartitions=1200)
    
    all_trips = all_trips.map_partitions(lambda x: x.sort_values('pickup_datetime'), 
        meta=all_trips)

    for fieldName in all_trips.columns:
        if fieldName in dtype_list:
            all_trips[fieldName] = all_trips[fieldName].astype(dtype_list[fieldName])
            
    all_trips.to_parquet(
        os.path.join(config['parquet_output_path'], 'all_trips_unprocessed.parquet'),
        compression='GZIP', 
#         has_nulls=True,
#         object_encoding='json'
    )


In [ ]:
%%time
if __name__ == '__main__':
    client = Client()

    # client=None
    # dask.set_options(get=dask.async.get_sync)

    main(client)

In [ ]:
from blazingsql import BlazingContext

bc = BlazingContext()

files = [f'data/all_trips_unprocessed.parquet/part.{i}.parquet' for i in range(2)]
bc.create_table('taxi', files)

bc.sql('select * from taxi')